# IS 4487 Assignment 11: Predicting Airbnb Prices with Regression

In this assignment, you will:
- Load the Airbnb dataset you cleaned and transformed in Assignment 7
- Build a linear regression model to predict listing price
- Interpret which features most affect price
- Try to improve your model using only the most impactful predictors
- Practice explaining your findings to a business audience like a host, pricing strategist, or city partner

## Why This Matters

Pricing is one of the most important levers for hosts and Airbnb’s business teams. Understanding what drives price — and being able to predict it accurately — helps improve search results, revenue management, and guest satisfaction.

This assignment gives you hands-on practice turning a cleaned dataset into a predictive model. You’ll focus not just on code, but on what the results mean and how you’d communicate them to stakeholders.

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Assignments/assignment_11_regression.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



## Original Source: Dataset Description

The dataset you'll be using is a **detailed Airbnb listing file**, available from [Inside Airbnb](https://insideairbnb.com/get-the-data/).

Each row represents one property listing. The columns include:

- **Host attributes** (e.g., host ID, host name, host response time)
- **Listing details** (e.g., price, room type, minimum nights, availability)
- **Location data** (e.g., neighborhood, latitude/longitude)
- **Property characteristics** (e.g., number of bedrooms, amenities, accommodates)
- **Calendar/booking variables** (e.g., last review date, number of reviews)

The schema is consistent across cities, so you can expect similar columns regardless of the location you choose.

In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


## 1. Load Your Transformed Airbnb Dataset

**Business framing:**  
Before building any models, we must start with clean, prepared data. In Assignment 7, you exported a cleaned version of your Airbnb dataset. You’ll now import that file for analysis.

### Do the following:
- Import your CSV file called `cleaned_airbnb_data_7.csv`.   (Note: If you had significant errors with assignment 7, you can use the file named "airbnb_listings.csv" in the DataSets folder on GitHub as a backup starting point.)
- Use `pandas` to load and preview the dataset

### In Your Response:
1. What does the dataset include?
2. How many rows and columns are present?


In [ ]:
# Add code here 🔧

In [62]:
df = pd.read_csv('listings (3).csv.gz')
display(df.head())

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5456,https://www.airbnb.com/rooms/5456,20250613040113,2025-06-13,city scrape,"Walk to 6th, Rainey St and Convention Ctr",Great central location for walking to Convent...,My neighborhood is ideally located if you want...,https://a0.muscache.com/pictures/14084884/b5a3...,8028,...,4.82,4.73,4.79,NaN,f,1,1,0,0,3.59
1,5769,https://www.airbnb.com/rooms/5769,20250613040113,2025-06-13,city scrape,NW Austin Room,NaN,Quiet neighborhood with lots of trees and good...,https://a0.muscache.com/pictures/23822033/ac94...,8186,...,4.94,4.77,4.92,NaN,f,1,0,1,0,1.65
2,6413,https://www.airbnb.com/rooms/6413,20250613040113,2025-06-14,previous scrape,Gem of a Studio near Downtown,"Great studio apartment, perfect a single perso...",Travis Heights is one of the oldest neighborho...,https://a0.muscache.com/pictures/hosting/Hosti...,13879,...,4.98,4.87,4.93,NaN,f,1,1,0,0,0.65
3,6448,https://www.airbnb.com/rooms/6448,20250613040113,2025-06-13,city scrape,"Secluded Studio @ Zilker - King Bed, Bright & ...","Clean, private space with everything you need ...",The neighborhood is fun and funky (but quiet)!...,https://a0.muscache.com/pictures/airflow/Hosti...,14156,...,4.98,4.97,4.88,NaN,t,1,1,0,0,2.02
4,8502,https://www.airbnb.com/rooms/8502,20250613040113,2025-06-13,city scrape,Woodland Studio Lodging,Studio rental on lower level of home located i...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,25298,...,4.88,4.69,4.63,NaN,f,1,1,0,0,0.29


### ✍️ Your Response: 🔧
1. The dataset includes all the data collected about specific airbnb rentals.
2. There is a max of 15,187 values and there are 79 columns present.

## 2. Drop Columns Not Useful for Modeling

**Business framing:**  
Some columns — like post IDs or text — may not help us predict price and could add noise or bias.

### Do the following:
- Drop columns like `post_id`, `title`, `descr`, `details`, and `address` if they’re still in your dataset

### In Your Response:
1. What columns did you drop, and why?
2. What risks might occur if you included them in your model?


In [63]:
# Step 1: clean all the data FIRST
# Drop columns with all missing values
df.dropna(axis=1, how='all', inplace=True)

# Fill missing values in numeric columns with the mean
numeric_cols = df.select_dtypes(include=['number']).columns
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].mean())

display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15187 entries, 0 to 15186
Data columns (total 76 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            15187 non-null  int64  
 1   listing_url                                   15187 non-null  object 
 2   scrape_id                                     15187 non-null  int64  
 3   last_scraped                                  15187 non-null  object 
 4   source                                        15187 non-null  object 
 5   name                                          15187 non-null  object 
 6   description                                   14840 non-null  object 
 7   neighborhood_overview                         7914 non-null   object 
 8   picture_url                                   15186 non-null  object 
 9   host_id                                       15187 non-null 

None

In [64]:
# Identify object columns to drop (excluding 'price')
object_cols = df.select_dtypes(include=['object']).columns.tolist()
columns_to_keep = ['price']
columns_to_drop_objects = [col for col in object_cols if col not in columns_to_keep]

# Add additional columns to drop as requested
additional_cols_to_drop = ['id','amenities','scrape_id', 'host_id', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm']

# Combine the lists of columns to drop
columns_to_drop = columns_to_drop_objects + additional_cols_to_drop

# Ensure 'license' is not in the list if it was already dropped
if 'license' in columns_to_drop and 'license' not in df.columns:
    columns_to_drop.remove('license')

# Drop columns that exist in the DataFrame
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
if existing_columns_to_drop:
    print(f"Dropping columns: {existing_columns_to_drop}")
    df.drop(columns=existing_columns_to_drop, inplace=True)
else:
    print("No additional columns to drop found in the DataFrame.")


display(df.head())

Dropping columns: ['listing_url', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'property_type', 'room_type', 'bathrooms_text', 'amenities', 'has_availability', 'calendar_last_scraped', 'first_review', 'last_review', 'instant_bookable', 'id', 'amenities', 'scrape_id', 'host_id', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm']


,host_listings_count,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1.0,2.0,78702,30.26057,-97.73441,3,1.00000,1.000000,2.000000,$101.00,...,4.86,4.90,4.82,4.73,4.79,1,1,0,0,3.59
1,1.0,4.0,78729,30.45697,-97.78422,2,1.00000,1.000000,1.000000,$45.00,...,4.87,4.91,4.94,4.77,4.92,1,0,1,0,1.65
2,1.0,1.0,78704,30.24885,-97.73587,2,1.73838,2.095101,2.972046,NaN,...,4.99,4.99,4.98,4.87,4.93,1,1,0,0,0.65
3,1.0,2.0,78704,30.26034,-97.76487,2,1.00000,1.000000,2.000000,$155.00,...,4.96,4.99,4.98,4.97,4.88,1,1,0,0,2.02
4,1.0,1.0,78741,30.23466,-97.73682,2,1.00000,1.000000,1.000000,$43.00,...,4.67,4.85,4.88,4.69,4.63,1,1,0,0,0.29


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15187 entries, 0 to 15186
Data columns (total 35 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_listings_count                           15187 non-null  float64
 1   host_total_listings_count                     15187 non-null  float64
 2   neighbourhood_cleansed                        15187 non-null  int64  
 3   latitude                                      15187 non-null  float64
 4   longitude                                     15187 non-null  float64
 5   accommodates                                  15187 non-null  int64  
 6   bathrooms                                     15187 non-null  float64
 7   bedrooms                                      15187 non-null  float64
 8   beds                                          15187 non-null  float64
 9   price                                         10708 non-null 

### ✍️ Your Response: 🔧
1. I droped alot of names, id # and oject columns. These  were identifiers like names and long descriptions which would be hard to use for regression analysis.
2. Risks that could occur is the data would not be repersentitive of the actual data, there was a lot of noise that needed to be cleaned up.

## 3. Explore Relationships Between Numeric Features

**Business framing:**  
Understanding how features relate to each other — and to the target — helps guide feature selection and modeling.

### Do the following:
- Generate a correlation matrix
- Identify which variables are strongly related to `price`

### In Your Response:
1. Which variables had the strongest positive or negative correlation with price?
2. Which variables might be useful predictors?


In [66]:
# Ensure 'price' is numeric (if not already)
if df['price'].dtype == 'object':
    df['price'] = df['price'].astype(str).str.replace('[$,]', '', regex=True)
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
    df.dropna(subset=['price'], inplace=True)

In [67]:
from sklearn.preprocessing import StandardScaler

# Identify all numerical columns
numerical_cols = df.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()

if numerical_cols:
    scaler = StandardScaler()
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    print("Scaled numerical columns:")
    print(numerical_cols)
else:
    print("No numerical columns to scale.")

display(df.head())

Scaled numerical columns:
['host_listings_count', 'host_total_listings_count', 'neighbourhood_cleansed', 'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'availability_eoy', 'number_of_reviews_ly', 'estimated_occupancy_l365d', 'estimated_revenue_l365d', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'reviews_per_month']


,host_listings_count,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,-0.196469,-0.207684,-1.109354,-0.317709,0.252042,-0.632529,-0.706133,-0.733113,-0.326324,-0.108955,...,0.143517,0.017072,-0.319617,-0.339517,0.043737,-0.485133,-0.432004,-0.228309,-0.093138,1.129947
1,-0.196469,-0.204990,0.187943,2.654900,-0.494204,-0.907681,-0.706133,-0.733113,-0.661906,-0.130329,...,0.176504,0.057568,0.159968,-0.191637,0.461942,-0.485133,-0.487873,-0.057802,-0.093138,-0.048753
3,-0.196469,-0.207684,-1.013258,-0.321190,-0.204305,-0.907681,-0.706133,-0.733113,-0.326324,-0.088345,...,0.473389,0.381537,0.319830,0.547763,0.333263,-0.485133,-0.432004,-0.228309,-0.093138,0.176051
4,-0.196469,-0.209032,0.764519,-0.709869,0.215936,-0.907681,-0.706133,-0.733113,-0.661906,-0.131092,...,-0.483239,-0.185408,-0.079825,-0.487397,-0.470977,-0.485133,-0.432004,-0.228309,-0.093138,-0.875058
5,-0.194623,-0.207684,-1.109354,-0.311503,0.307325,-0.632529,0.249302,-0.076992,-0.326324,-0.092162,...,0.407415,0.422033,0.399761,0.658673,0.526281,-0.436394,-0.376136,-0.228309,-0.093138,-0.990498


In [68]:
# Calculate the correlation matrix with 'price'
correlation_matrix = df_for_corr.corr()['price'].sort_values(ascending=False)

# Drop NaN values from the correlation matrix
correlation_matrix.dropna(inplace=True)

display(correlation_matrix)

,price
price,1.000000
host_total_listings_count,0.394259
estimated_revenue_l365d,0.366934
host_listings_count,0.299557
calculated_host_listings_count,0.209643
maximum_nights,0.106529
calculated_host_listings_count_private_rooms,0.058815
availability_30,0.052953
bathrooms,0.051160
availability_365,0.049873


### ✍️ Your Response: 🔧
1. The variables with the strongest positve correlations were host_total_listings_count, estimated_revenue_l365d, and host_listings_count. The variables with the strongest negative correlations were longitude, estimated_occupancy_l365d, and number_of_reviews_ltm.

2. The variables that could be useful predictors should be the ones with the highest correlations.

## 4. Define Features and Target Variable

**Business framing:**  
To build a regression model, you need to define what you’re predicting (target) and what you’re using to make that prediction (features).

### Do the following:
- Set `price` as your target variable
- Remove `price` from your predictors

### In Your Response:
1. What features are you using?
2. Why is this a regression problem and not a classification problem?


In [ ]:
# Add code here 🔧

In [69]:
# Define features (X) and target variable (y)
X = df.drop('price', axis=1)
y = df['price']

print("Features (X) shape:", X.shape)
print("Target variable (y) shape:", y.shape)

Features (X) shape: (10708, 34)
Target variable (y) shape: (10708,)


### ✍️ Your Response: 🔧
1. I used all the columns excluding price.

2. This is a regression problem because the target variable is a numerical value and the goal is to predict a price( numerical).

## 5. Split Data into Training and Testing Sets

### Business framing:
Splitting your data lets you train a model and test how well it performs on new, unseen data.

### Do the following:
- Use `train_test_split()` to split into 80% training, 20% testing



In [70]:
# Split data into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (8566, 34)
X_test shape: (2142, 34)
y_train shape: (8566,)
y_test shape: (2142,)


## 6. Fit a Linear Regression Model

### Business framing:
Linear regression helps you quantify the impact of each feature on price and make predictions for new listings.

### Do the following:
- Fit a linear regression model to your training data
- Use it to predict prices for the test set



In [ ]:
# Add code here 🔧

In [71]:
# Fit a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict prices for the test set
y_pred = model.predict(X_test)

print("Model fitting and prediction complete.")

Model fitting and prediction complete.


## 7. Evaluate Model Performance

### Business framing:  
A good model should make accurate predictions. We’ll use Mean Squared Error (MSE) and R² to evaluate how close our predictions were to the actual prices.

### Do the following:
- Print MSE and R² score for your model

### In Your Response:
1. What is your R² score? How well does your model explain price variation?
2. Is your MSE large or small? What could you do to improve it?


In [72]:
# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

Mean Squared Error (MSE): 0.23
R-squared (R²): 0.71


### ✍️ Your Response: 🔧
1. My r^2 was .71. This means that 71% of the variation in Airbnb prices can be explained by the features in this model. This is considered a pretty good fit.

2. My MSE is a bit high. A MSE at 0 is the ideal. To imporve it I would try and refine the model, and make sure there is no reiduals, or outliers.

## 8. Interpret Model Coefficients

### Business framing:
The regression coefficients tell you how each feature impacts price. This can help Airbnb guide hosts and partners.

### Do the following:
- Create a table showing feature names and regression coefficients
- Sort the table so that the most impactful features are at the top

### In Your Response:
1. Which features increased price the most?
2. Were any surprisingly negative?
3. What business insight could you draw from this?


In [ ]:
# Add code here 🔧

In [73]:
# Create a DataFrame of features and coefficients
coefficients_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model.coef_
})

# Sort the DataFrame by the absolute value of the coefficients
coefficients_df['Abs_Coefficient'] = coefficients_df['Coefficient'].abs()
coefficients_df = coefficients_df.sort_values(by='Abs_Coefficient', ascending=False).drop(columns='Abs_Coefficient')

display(coefficients_df)

,Feature,Coefficient
29,calculated_host_listings_count,3.029399
30,calculated_host_listings_count_entire_homes,-2.652028
31,calculated_host_listings_count_private_rooms,-0.871321
32,calculated_host_listings_count_shared_rooms,-0.808956
21,estimated_revenue_l365d,0.169411
6,bathrooms,0.064879
20,estimated_occupancy_l365d,-0.061942
22,review_scores_rating,-0.034984
2,neighbourhood_cleansed,-0.033274
7,bedrooms,-0.030608


### ✍️ Your Response: 🔧
1. calculated_host_listings_count increased the price the most.

2. calculated_host_listings_count_entire_homes was surprisingly negative.

3. A business insight from this is the negative coefficients for estimated_occupancy_l365d and review_scores_rating might show that listings with proven success (high occupancy, great reviews) are not using that that success to charge higher prices.


## 9. Try to Improve the Linear Regression Model

### Business framing:
The first version of your model included all available features — but not all features are equally useful. Removing weak or noisy predictors can often improve performance and interpretation.

### Do the following:
1. Choose your top 3–5 features with the strongest absolute coefficients
2. Rebuild the regression model using just those features
3. Compare MSE and R² between the baseline and refined model

### In Your Response:
1. What features did you keep in the refined model, and why?
2. Did model performance improve? Why or why not?
3. Which model would you recommend to stakeholders?
4. How does this relate to your customized learning outcome you created in canvas?


In [80]:
# Step 1: Choose top features based on absolute coefficients
# Using the coefficients_df generated in cell tZrS5A_V_-rE
top_n_features = 5 # You can adjust this number (3-5)
selected_features_refined = coefficients_df['Feature'].head(top_n_features).tolist()

print(f"Selected top {top_n_features} features: {selected_features_refined}")

# Step 2: Redefine features (X) using only the selected features
X_refined = df[selected_features_refined]
y_refined = df['price'] # Target variable remains the same

# Step 3: Split the data into training and testing sets again
X_train_refined, X_test_refined, y_train_refined, y_test_refined = train_test_split(X_refined, y_refined, test_size=0.2, random_state=42)

print("\nShapes of refined data splits:")
print("X_train_refined shape:", X_train_refined.shape)
print("X_test_refined shape:", X_test_refined.shape)
print("y_train_refined shape:", y_train_refined.shape)
print("y_test_refined shape:", y_test_refined.shape)

# Step 4: Rebuild and fit the regression model using just those features
model_refined = LinearRegression()
model_refined.fit(X_train_refined, y_train_refined)

# Step 5: Predict prices for the test set using the refined model
y_pred_refined = model_refined.predict(X_test_refined)

print("\nRefined model fitting and prediction complete.")

Selected top 5 features: ['calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'estimated_revenue_l365d']

Shapes of refined data splits:
X_train_refined shape: (8566, 5)
X_test_refined shape: (2142, 5)
y_train_refined shape: (8566,)
y_test_refined shape: (2142,)

Refined model fitting and prediction complete.


In [79]:
# Evaluate performance of the refined model
mse_refined = mean_squared_error(y_test_refined, y_pred_refined)
r2_refined = r2_score(y_test_refined, y_pred_refined)

print(f"Refined Model Mean Squared Error (MSE): {mse_refined:.2f}")
print(f"Refined Model R-squared (R²): {r2_refined:.2f}")

Refined Model Mean Squared Error (MSE): 0.23
Refined Model R-squared (R²): 0.71


### ✍️ Your Response: 🔧
1. I kept the top 5 features in the refined model.

2. Preformance stayed the same. This means the final set of predictions ws the same form the predictions of my previous model.
3. I would reccomend the refined model because it's easier to interpret and esier to figure out what needs to be inproved.
4. This relates to my costomising learning outcome of sustainable operations problem-solving by using data to improve efficiency and make better decisions. Specifically, getting the same r^2/MSE with fewer features is an example of Supply Chain Sustainability Optimization by reducing operational complexity and waste while still maintaining performance.

## 10. Reflect and Recommend

### Business framing:  
Ultimately, the value of your model comes from how well it can guide business decisions. Use your results to make real-world recommendations.

### In Your Response:
1. What business question did your model help answer?
2. What would you recommend to Airbnb or its hosts?
3. What could you do next to improve this model or make it more useful?
4. How does this relate to your customized learning outcome you created in canvas?


### ✍️ Your Response: 🔧
1. The model helped answer what features have the gratest impact on the airbnb prices in Austin, TX.

2. To the hosts I would reccomend them prioritize their portfolio. Theyshould focus less on expensive, minor physical improvements and more on their overall listing strategy. The coefficents showed that managing the mix of entire homes, private rooms, and shared rooms is the primary lever for price control. For Airbnb they can make a targeted pricing tool using the identified 3-5 core features to build a simpler, faster, and highly accurate price recommendation engine. This model would be more accurate and easier to explain to hosts than one based on dozens of features.

3. To improve this model, I could figure out how to make some of the long review descriptions into analyzable data. This will allow me to figure out if there are any other factors that could affect the price that were not recongnized in this anlaysis.

4. This relates to my customized learning outcomes of 1. Supply Chain Sustainability Optimization by showing that you can maintain performance with fewer features which is directly related to reducing waste and optimizing efficiency in an operation.2.Sustainable Operations Problem-Solving Framework because I turned a real-world market challenge into a measurable business problem  and then utilized the data to figure out actionable insights.

## Submission Instructions
✅ Checklist:
- All code cells run without error
- All markdown responses are complete
- Submit on Canvas as instructed

In [81]:
!jupyter nbconvert --to html "assignment_11_LastnameFirstname.ipynb"

[NbConvertApp] WARNING | pattern 'assignment_11_LastnameFirstname.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes